https://zh.wikipedia.org/wiki/%E5%81%9C%E7%94%A8%E8%AF%8D<span class="mark">停用词的网站</span>

多项式与伯努利的区别就是，每个向量化的文本是有计数的

In [118]:
import numpy as np
import re

In [119]:
#首先找出所有不重复的词语作为特征
def uniqueword(datalist):
    wordlist=set([])
    for i in datalist:
        wordlist=wordlist| set(i)#必须是集合与集合之间的并集
    return list(wordlist)

In [120]:
#将每一个列表的文本向量化
def wordvec(wordlist,unit_list):
    original=np.zeros(len(wordlist))
    for word in unit_list:
        if word in wordlist:
            original[wordlist.index(word)]+=1#这一步是最大的区别
    return original

In [121]:
#解析导入的文本，正则化，除去非字符串
def textParse(txt):
    import re
    txt_split=re.split(r'\W+',txt)
    return [i.lower() for i in txt_split if len(i)>2]#排除了字符串长度小于等于2的

In [122]:
#计算条件概率
def prob(word,classlist,wordlist):
    #计算垃圾邮件的概率
    p=sum(classlist)/len(classlist)
    #初始化
    original_1=np.ones(len(wordlist))#每个文本的计数
    original_0=np.ones(len(wordlist))
    sum1=2#初始化该类别总单词数为2
    sum0=2
    for i in range(len(word)):
        if classlist[i]==1:
            original_1+=wordvec(wordlist,word[i])
            sum1+=sum(wordvec(wordlist,word[i]))#种类为1的
        else:
            original_0+=wordvec(wordlist,word[i])
            sum0+=sum(wordvec(wordlist,word[i]))#种类为0的
        
    return np.log(original_1/sum1),np.log(original_0/sum0),p

In [123]:
#多项式贝叶斯分类器
def MultinomialNB(wordvec,p0,p1,p_class1):
    word_class1=sum(wordvec*p1)+np.log(p_class1)
    word_class0=sum(wordvec*p0)+np.log(1-p_class1)
    if word_class1>word_class0:
        return 1
    else:
        return 0

In [129]:
#导入数据，并计算他的条件概率
def analyse():
    doclist=[]
    classlist=[]
    for i in range(1,25):#一共48封
        data_spam=textParse(open(r'E:\贝叶斯垃圾邮件分类数据\Training\spam\%d.txt' %i).read())#把该文件夹下所有的txt读取
        doclist.append(data_spam)
        classlist.append(1)
    
        data_normal=textParse(open(r'E:\贝叶斯垃圾邮件分类数据\Training\normal\%d.txt' %i).read())
        doclist.append(data_normal)
        classlist.append(0)
        
    wordlist=uniqueword(doclist)#生成去重后的所有单词,返回一个list
    
    #完成数据集后，将数据集进行cross validation交叉验证，即随机取出8组list作为验证
    #不好根据索引批量删除验证集的数据，这里选择进行循环找随机数
    doclist_cv=[]
    classlist_cv=[]
    import random
    for i in range(8):#弄8个验证集
        indice=random.randint(0,len(doclist))
        doclist_cv.append(doclist[indice])#验证集文本
        classlist_cv.append(classlist[indice])#验证集分类
        del(doclist[indice])
        del(classlist[indice])
    
    #此时doclist就是训练集文本,classlist就是训练集文本的分类
    doclist_trainvec=[]
    for unit_txt in doclist:
        #将训练集中每个文本进行向量化
        doclist_trainvec.append(wordvec(wordlist,unit_txt))
    
    
    print(doclist_trainvec)
    #接下来生成每个单词的条件概率
    p1,p0,p_class1=prob(doclist_trainvec,classlist,wordlist)
    print(p1)
    
    #将训练集生成的条件概率带入多项式朴素贝叶斯,测试验证集正确率
    doclist_cv_vec=[]
    for unit_txt in doclist_cv:
        doclist_cv_vec.append(wordvec(wordlist,unit_txt))#向量化验证集
    
    error=0
    for test in range(len(doclist_cv_vec)):
        if MultinomialNB(np.array(doclist_cv_vec[test]),p0,p1,p_class1)!= classlist_cv[test]:#第一个参数需要向量化
            error+=1
    
    return error/len(classlist_cv)#指~accuracy

In [130]:
#测试
analyse()

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0

[-0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0.69314718
 -0.69314718 -0.69314718 -0.69314718 -0.69314718 -0

0.75

2